In [15]:
from pandas_datareader import data
import datetime
from datetime import timedelta
import jpholiday
import operator
import pandas as pd
import numpy as np
import warnings

warnings.resetwarnings()
warnings.simplefilter('ignore', pd.errors.SettingWithCopyWarning)

#ref. : https://qiita.com/hid_tanabe/items/3c5e6e85c6c65f7b38be
def isBizDay(DATE):
    Date = datetime.date(int(DATE.year), int(DATE.month), int(DATE.day))
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1

def n_bizdays_ago(n): #n営業日前を出す
    #stooqの仕様的に、翌営業日にならないと前営業日のデータは取れない。
    #よって今日を含んでn+1日ぶん数えなくてはならない。例えば日曜日の１営業日前は木曜日になる。
    today = datetime.datetime.now()
    i = 0 #結果的に休日を含んだn日前になるカウンター
    c = n+1 #平日を数えた時にだけ減らしていくカウンター
    while c!=0:
        if isBizDay(today - timedelta(i)): #i日前が平日だったら
            n_bizdays = today - timedelta(i)
            i+=1
            c-=1
        else:
            i+=1
    return n_bizdays

def stock_pl(df_purchase, df_company): #買った日付、買った銘柄（証券コード）,買った数　の列を含むdf
    pl_sum = 0
    pl_sum_fee = 0
    df_purchase_pl = df_purchase #損益用に新しいdf
    #df_purchase_pl.insert(2, 'Company', '---')#会社名用
    df_purchase_pl['unit_buy'] = 0 #終値単価ー勝った時
    df_purchase_pl['unit_yes'] = 0 #終値単価ー昨日
    df_purchase_pl['buy'] = 0 #買った日の終値いくら　列追加
    df_purchase_pl['yesterday'] = 0 #昨日の終値いくら　列追加
    df_purchase_pl['P/L'] = 0 #昨日の終値時点での損益いくら　列追加
    df_purchase_pl['fee_buy'] = 0 #買った時の委託手数料
    df_purchase_pl['fee_sell_yes'] = 0 #昨日時点で売ってた場合の委託手数料
    df_purchase_pl['P/L(in_fee)'] = 0 #手数料含めて、昨日の終値時点で売った場合の損益
    #df_purchase_pl.loc['sum'] = 
    for i in range(len(df_purchase)):
        #df_purchase_pl['Company'][i] = df_company.query('コード == df_purchase["Code"][i]')['銘柄名'][0]
        #stooqのデータフレームの順番は、後の日付→→前の日付　買った日→昨日とするなら、昨日が一番上に来る。勝った日は一番下。
        df_purchase_buy2yest = data.DataReader(df_purchase_pl['Code'][i].astype('str') + '.JP', 'stooq', datetime.datetime.strptime(df_purchase['Date'][i], "%Y-%m-%d"), n_bizdays_ago(1))
        df_purchase_pl['unit_buy'][i] = df_purchase_buy2yest.iloc[-1]['Close']
        df_purchase_pl['unit_yes'][i] = df_purchase_buy2yest['Close'][0]
        df_purchase_pl['buy'][i] = df_purchase_buy2yest.iloc[-1]['Close'] * df_purchase_pl['Amount'][i]
        df_purchase_pl['yesterday'][i] = df_purchase_buy2yest['Close'][0] * df_purchase_pl['Amount'][i]
        df_purchase_pl['P/L'][i] = df_purchase_pl['yesterday'][i] - df_purchase_pl['buy'][i]
        df_purchase_pl['fee_buy'][i] = df_purchase_pl['buy'][i] * 0.005 * 1.10
        df_purchase_pl['fee_sell_yes'][i] = df_purchase_pl['yesterday'][i] * 0.005 * 1.10
        df_purchase_pl['P/L(in_fee)'][i] = df_purchase_pl['P/L'][i] - df_purchase_pl['fee_buy'][i] - df_purchase_pl['fee_sell_yes'][i]
        pl_sum += df_purchase_pl['P/L'][i]
        pl_sum_fee += df_purchase_pl['P/L(in_fee)'][i]

    print(df_purchase_pl)
    print(pl_sum)
    print(pl_sum_fee)


def stock_bought_rank(df_purchase):
    twodays_ago = n_bizdays_ago(2)
    yesterday = n_bizdays_ago(1)
    df_ranking = pd.DataFrame(np.arange(4*len(df_purchase)).reshape([len(df_purchase), 4]))#ランキング用のdf
    #df_ranking.set_axis(['Code', '2days_ago', '1day_ago', 'Diff'], axis=1)#列名をセット→失敗
    df_ranking.columns = ['Code', '2days_ago', '1day_ago', 'Diff']
    for i in range(len(df_purchase)):
        df_ranking['Code'][i] = df_purchase['Code'][i]
        df_temp = data.DataReader(df_ranking['Code'][i].astype('str') + '.JP', 'stooq', twodays_ago, yesterday)
        df_ranking['2days_ago'][i] = df_temp['Close'][1]#2営業日前の終値
        df_ranking['1day_ago'][i] = df_temp['Close'][0]#1営業日前の終値
        df_ranking['Diff'][i] = df_temp['Close'][0] - df_temp['Close'][1]#↑2つの差

    df_sorted = df_ranking.sort_values('Diff', ascending=False)#ソート(降順)
    print(df_sorted)


df1 = pd.read_csv('stock_purchase_data.csv', header=0)
df2 = pd.read_csv('data_j.csv', header=0)
stock_pl(df1, df2)
stock_bought_rank(df1)


#展望
#手数料の反映
#差の変動(微分?)をグラフとかにしたい

<ipython-input-15-36578bb0c0b5>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchase_pl['unit_buy'][i] = df_purchase_buy2yest.iloc[-1]['Close']
<ipython-input-15-36578bb0c0b5>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchase_pl['unit_yes'][i] = df_purchase_buy2yest['Close'][0]
<ipython-input-15-36578bb0c0b5>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_purchase_pl['buy'][i] = df_purchase_buy2yest.iloc[-1][

         Date  Code  Amount  unit_buy  unit_yes      buy  yesterday    P/L  \
0  2023-05-24  7751     100      3428      3500   342800     350000   7200   
1  2023-05-24  9021     200      5939      5820  1187800    1164000 -23800   
2  2023-05-24  9020     200      7806      7693  1561200    1538600 -22600   
3  2023-05-24  7974     100      5930      5954   593000     595400   2400   
4  2023-05-31  6326     100      1927      1930   192700     193000    300   
5  2023-05-31  7267     100      4005      3978   400500     397800  -2700   

   fee_buy  fee_sell_yes  P/L(in_fee)  
0     1885          1925         3390  
1     6532          6402       -36734  
2     8586          8462       -39648  
3     3261          3274        -4135  
4     1059          1061        -1820  
5     2202          2187        -7089  
-39200
-86036
   Code  2days_ago  1day_ago  Diff
0  7751       3461      3500    39
3  7974       5936      5954    18
2  9020       7688      7693     5
4  6326       1927 

In [21]:
from pandas_datareader import data
import datetime
from datetime import timedelta
import jpholiday
import operator
import pandas as pd
import numpy as np
import sys
import warnings

warnings.resetwarnings()
warnings.simplefilter('ignore')

#ref. : https://qiita.com/hid_tanabe/items/3c5e6e85c6c65f7b38be
def isBizDay(DATE):
    Date = datetime.date(int(DATE.year), int(DATE.month), int(DATE.day))
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1

def n_bizdays_ago(n): #n営業日前を出す
    #stooqの仕様的に、翌営業日にならないと前営業日のデータは取れない。
    #よって今日を含んでn+1日ぶん数えなくてはならない。例えば日曜日の１営業日前は木曜日になる。
    today = datetime.datetime.now()
    i = 0 #結果的に休日を含んだn日前になるカウンター
    c = n+1 #平日を数えた時にだけ減らしていくカウンター
    while c!=0:
        if isBizDay(today - timedelta(i)): #i日前が平日だったら
            n_bizdays = today - timedelta(i)
            i+=1
            c-=1
        else:
            i+=1
    return n_bizdays

def stock_pl(df_purchase, hold_money): #売買日付、売か買か、売買した銘柄（証券コード）,売買した数　の列を含むdf
    pl_sum = hold_money
    #pl_sum_fee = 0
    df_purchase_pl = df_purchase #損益用に新しいdf
    df_holding_stock =pd.DataFrame({ 'Code': [0],
                                     'Amount': [0]}) #持ち株データを保持(先頭にダミーデータ)
    #df_purchase_pl.insert(2, 'Company', '---')#会社名用
    df_purchase_pl['unit_purchase'] = 0 #終値単価ー売買時
    df_purchase_pl['purchase'] = 0 #売買した日の終値いくら　列追加
    #df_purchase_pl['P/L'] = 0 #昨日の終値時点での損益いくら　列追加
    df_purchase_pl['fee'] = 0 #売買時の委託手数料
    #df_purchase_pl['P/L(in_fee)'] = 0 #手数料含めて、昨日の終値時点で売った場合の損益
    #df_purchase_pl.loc['sum'] = 
    for i in range(len(df_purchase)):
        if df_purchase_pl['buy_or_sell'][i] == 0: #買ったデータのとき
            #df_purchase_pl['Company'][i] = df_company.query('コード == df_purchase["Code"][i]')['銘柄名'][0]
            purchase_day = datetime.datetime.strptime(df_purchase['Date'][i], "%Y-%m-%d") #買った日
            #print(purchase_day)
            df_purchase_realdata = data.DataReader(df_purchase_pl['Code'][i].astype('str') + '.JP', 'stooq', purchase_day, purchase_day)
            df_purchase_pl['unit_purchase'][i] = df_purchase_realdata['Close'][0] #買った株の単価
            df_purchase_pl['purchase'][i] = - (df_purchase_realdata['Close'][0] * df_purchase_pl['Amount'][i])
            df_purchase_pl['fee'][i] = (df_purchase_pl['purchase'][i] * 0.005 * 1.10)
            pl_sum += (df_purchase_pl['purchase'][i] + df_purchase_pl['fee'][i])
            

            df_temp = df_holding_stock[df_holding_stock['Code'] == df_purchase_pl['Code'][i]] #いま見ている売買データの銘柄が保持株にあるか検索してその行を抜き出す
            if not df_temp.empty: #すでに買っている株を追加で買った場合
                df_holding_stock['Amount'][df_temp.index.values[0]] += df_purchase_pl['Amount'][i]
                print(df_holding_stock)
            else: #いま保持していない株を買った場合、保有株データに登録
                df_holding_stock.loc[len(df_holding_stock)] = [df_purchase_pl['Code'][i], df_purchase_pl['Amount'][i]]
                print(df_holding_stock)

        else: #売ったデータの時
            purchase_day = datetime.datetime.strptime(df_purchase['Date'][i], "%Y-%m-%d") #売った日
            #print(purchase_day)
            df_purchase_realdata = data.DataReader(df_purchase_pl['Code'][i].astype('str') + '.JP', 'stooq', purchase_day, purchase_day)
            #print(df_purchase_realdata)
            df_purchase_pl['unit_purchase'][i] = df_purchase_realdata['Close'][0] #売った株の単価
            df_purchase_pl['purchase'][i] = df_purchase_realdata['Close'][0] * df_purchase_pl['Amount'][i] #売った株の価格
            df_purchase_pl['fee'][i] = - (df_purchase_pl['purchase'][i] * 0.005 * 1.10)
            pl_sum += (df_purchase_pl['purchase'][i] + df_purchase_pl['fee'][i])

            df_temp =  df_holding_stock[df_holding_stock['Code'] == df_purchase_pl['Code'][i]] #いま見ている売買データの銘柄が保持株にあるか検索してその行を抜き出す
            if df_temp.empty: #持っていないのに売ったことになっている場合
                print('パターン1')
                sys.exit('Invalid stock-trade data in the row' + str(i))
            else:
                index_temp = df_temp.index.values[0] #保持株dfのうちいま見ている銘柄の行のインデックス
                print('index_temp:' + str(index_temp))
                stockAmount_diff = df_holding_stock['Amount'][index_temp] - df_purchase_pl['Amount'][i] #保持している株-売った株数
                if stockAmount_diff == 0: #この株売りによってその会社の株が無くなった場合（＝全部売った）
                    df_holding_stock = df_holding_stock.drop(index_temp) #保持株データからその行を消す
                    print(df_holding_stock)
                    df_holding_stock = df_holding_stock.set_axis(list(range(len(df_holding_stock))))
                    print(df_holding_stock)
                elif stockAmount_diff > 0: #部分的に売った場合
                    df_holding_stock['Amount'][index_temp] -= df_purchase_pl['Amount'][i]
                else: #保持株以上に株を売ったようなデータになっているとき
                    print('パターン2')
                    sys.exit('Invalid stock-trade data in the row' + str(i))
        
        #print(i)
    
    df_holding_stock = df_holding_stock.set_axis(list(range(len(df_holding_stock)))) #index(行名)を振り直す
    holding_stock_value = 0
    yesterday = n_bizdays_ago(1)
    for i in range(1, len(df_holding_stock)):
        df_temp = data.DataReader(df_holding_stock['Code'][i].astype('str') + '.JP', 'stooq', yesterday, yesterday)
        print(df_temp)
        holding_stock_value += df_holding_stock['Amount'][i] * df_temp['Close'][0]

    print(df_purchase_pl)
    print("Holding_money: " + str(pl_sum)) #いま持っている所持金
    print("Holding_stock: " + str(holding_stock_value)) #いま持っている株の価値
    total_assets = pl_sum + holding_stock_value
    print("Total_assets: " + str(total_assets))
    capital_gains_tax = 0 #譲渡益税（現在の儲け(=所持金+所持株の価値が1000万を超える場合、超えた分の20%)）
    if total_assets > 10000000:
        capital_gains_tax = ((total_assets) - 10000000) * 0.2
    print("capital_gains_tax: " + str(capital_gains_tax))


def stock_bought_rank(df_purchase):
    twodays_ago = n_bizdays_ago(2)
    yesterday = n_bizdays_ago(1)
    df_ranking = pd.DataFrame(np.arange(4*len(df_purchase)).reshape([len(df_purchase), 4]))#ランキング用のdf
    #df_ranking.set_axis(['Code', '2days_ago', '1day_ago', 'Diff'], axis=1)#列名をセット→失敗
    df_ranking.columns = ['Code', '2days_ago', '1day_ago', 'Diff']
    for i in range(len(df_purchase)):
        df_ranking['Code'][i] = df_purchase['Code'][i]
        df_temp = data.DataReader(df_ranking['Code'][i].astype('str') + '.JP', 'stooq', twodays_ago, yesterday)
        df_ranking['2days_ago'][i] = df_temp['Close'][1]#2営業日前の終値
        df_ranking['1day_ago'][i] = df_temp['Close'][0]#1営業日前の終値
        df_ranking['Diff'][i] = df_temp['Close'][0] - df_temp['Close'][1]#↑2つの差

    df_sorted = df_ranking.sort_values('Diff', ascending=False)#ソート(降順)
    print(df_sorted)


df1 = pd.read_csv('stock_trade_data.csv', header=0)
df2 = pd.read_csv('data_j.csv', header=0)
stock_pl(df1, 10000000)
#stock_bought_rank(df1)


#展望
#手数料の反映
#差の変動(微分?)をグラフとかにしたい

   Code  Amount
0     0       0
1  7751     100
   Code  Amount
0     0       0
1  7751     100
2  9021     200
   Code  Amount
0     0       0
1  7751     100
2  9021     200
3  9020     200
   Code  Amount
0     0       0
1  7751     100
2  9021     200
3  9020     200
4  7974     100
index_temp:1
   Code  Amount
0     0       0
2  9021     200
3  9020     200
4  7974     100
   Code  Amount
0     0       0
1  9021     200
2  9020     200
3  7974     100
index_temp:1
   Code  Amount
0     0       0
1  9021     100
2  9020     200
3  7974     100
4  6326     100
   Code  Amount
0     0       0
1  9021     100
2  9020     200
3  7974     100
4  6326     100
5  7267     100
   Code  Amount
0     0       0
1  9021     100
2  9020     200
3  7974     100
4  6326     200
5  7267     100
            Open  High   Low  Close  Volume
Date                                       
2023-06-19  5987  5987  5912   5948  837600
            Open  High   Low  Close  Volume
Date                          

In [8]:
from pandas_datareader import data
import datetime
from datetime import timedelta
import jpholiday
import pandas as pd

df1 = pd.read_csv('stock_purchase_data.csv', header=0)
#df1.set_index(['Date'])
print(df1)
print(df1['Date'][0])
print(df1.tail(1)['Code'])
print(df1.iloc[-1]['Code'])
print(len(df1))

#ref. : https://qiita.com/hid_tanabe/items/3c5e6e85c6c65f7b38be
def isBizDay(DATE):
    Date = datetime.date(int(DATE.year), int(DATE.month), int(DATE.day))
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 0
    else:
        return 1

def n_bizdays_ago_old(n): #n営業日前を出す
    today = datetime.datetime.now()
    i = 1
    c = n
    while c!=0:
        if isBizDay(today - timedelta(i)): #n日前が平日になるまで遡る
            n_bizdays = today - timedelta(i)
            i+=1
            c-=1
        else:
            i+=1
    return n_bizdays


def n_bizdays_ago(n): #n営業日前を出す
    #stooqの仕様的に、翌営業日にならないと前営業日のデータは取れない。
    #よって今日を含んでn+1日ぶん数えなくてはならない。例えば日曜日の１営業日前は木曜日になる。
    today = datetime.datetime.now()
    i = 0 #結果的に休日を含んだn日前になるカウンター
    c = n+1 #平日を数えた時にだけ減らしていくカウンター
    while c!=0:
        if isBizDay(today - timedelta(i)): #n日前が平日だったら
            n_bizdays = today - timedelta(i)
            i+=1
            c-=1
        else:
            i+=1
    return n_bizdays

df2 = pd.DataFrame({'Reimu': [0,1,2],
                    'Marisa': [3,4,5],
                    'Youmu': [6,7,8]})

print(df2[df2['Reimu'] == 1].index.values[0])
df3 = df2.drop(df2.index[1])
print(list(range(len(df3))))
df3 = df3.set_axis(list(range(len(df3))))
print(df3)
print(len(df3))
df3.loc[len(df3)] = [11, 12, 13]
print(len(df3))
print(df3)
#print(df3['Marisa'][1])

df4 = data.DataReader('7751.JP', 'stooq', n_bizdays_ago(1), n_bizdays_ago(1))
print(df4['Close'][0])

print(n_bizdays_ago(1))
print(n_bizdays_ago(2))
print(n_bizdays_ago(10))
df1 = pd.read_csv('stock_purchase_data.csv', header=0)
print(len(df1))

         Date  Code  Amount
0  2023-05-24  7751     100
1  2023-05-24  9021     200
2  2023-05-24  9020     200
3  2023-05-24  7974     100
4  2023-05-31  6326     100
5  2023-05-31  7267     100
2023-05-24
5    7267
Name: Code, dtype: int64
7267
6
1
[0, 1]
   Reimu  Marisa  Youmu
0      0       3      6
1      2       5      8
2
3
   Reimu  Marisa  Youmu
0      0       3      6
1      2       5      8
2     11      12     13
3807
2023-06-19 09:43:08.251831
2023-06-16 09:43:08.252314
2023-06-06 09:43:08.252862
6
